### Packages

In [2]:
import ast
import concurrent.futures
import csv
import cv2
import datetime
import dateutil.parser
import distutils.dir_util
import gc
import glob
import IPython.display
import ipywidgets as widgets
import joblib
import json
import keras
import math
import matplotlib.pyplot as plt
import numpy as np
import openmeteo_requests
import os
import pandas as pd
import pathlib
import pickle
import polars as pl
import plotly.express as px
import pyautogui
import pyperclip
import pytz
import re
import requests
import requests_cache
from retry_requests import retry
import selenium
import shutil
import seaborn as sns
import sqlite3
import statsapi
import statsmodels.formula.api as smf
import statsmodels.api as sm
import subprocess
import sys
import tensorflow as tf
import time
import traceback
import unidecode
import warnings
import webbrowser
import xlrd
import random
import urllib
import zipfile

from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from copy import deepcopy
from dateutil import parser

from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

from functools import partial
from io import StringIO
from IPython.display import display, Javascript, clear_output
from joblib import Parallel, delayed
from lxml import html
from openpyxl import load_workbook
from paretoset import paretoset
from pathlib import Path
from pulp import GLPK_CMD  
from pybaseball import statcast
from pydfs_lineup_optimizer import get_optimizer, Site, Sport, Player, TeamStack, PlayerFilter, RandomFantasyPointsStrategy, ProgressiveFantasyPointsStrategy, AfterEachExposureStrategy, LineupOptimizer
# from pydfs_lineup_optimizer.solvers.mip_solver import MIPSolver
from pydfs_lineup_optimizer.solvers import PuLPSolver

from scipy import stats
from scipy.stats import ttest_ind, mannwhitneyu
from scipy.stats.mstats import winsorize

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium import webdriver


from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
import smtplib
import ssl
from statsapi import get
from sqlalchemy import create_engine
from tensorflow.keras.losses import KLDivergence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tqdm import tqdm


from urllib.request import urlopen, Request



from webdriver_manager.chrome import ChromeDriverManager

# Ensure the warning is ignored only once
warnings.simplefilter(action="ignore")

# Display the DataFrame
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

# Set paths
model_path = r"C:\Users\james\Documents\MLB\Models"
baseball_path = r"C:\Users\james\Documents\MLB\Database"
download_path = r"C:\Users\james\Downloads"

### Dates

In [ ]:
# Today's Date
# YYYY-MM-DD (datetime)
todaysdate_dt = datetime.date.today()

# YYYY-MM-DD (string)
todaysdate_dash = str(todaysdate_dt)

# MM/DD/YYYY
todaysdate_slash = todaysdate_dash.split("-")
todaysdate_slash = todaysdate_slash[1] + "/" + todaysdate_slash[2] + "/" + todaysdate_slash[0]

# YYYYMMDD
todaysdate = todaysdate_dash.replace("-", "")

## MM-DD-YYYY
todaysdate_dash = todaysdate[:4] + "-" + todaysdate[4:6] + "-" + todaysdate[6:]

In [ ]:
# Get the current date
current_date = datetime.datetime.now()

# Subtract one day from the current date to get yesterday's date
yesterday_dt = current_date - datetime.timedelta(days=1)

# Format yesterday's date as "YYYYMMDD"
yesterdaysdate = yesterday_dt.strftime("%Y%m%d")

# MM/DD/YYYY
yesterdaysdate_slash = yesterdaysdate[4:6] + "/" + yesterdaysdate[6:8] + "/" + yesterdaysdate[0:4] 

## MM-DD-YYYY
yesterdaysdate_dash = yesterdaysdate[:4] + "-" + yesterdaysdate[4:6] + "-" + yesterdaysdate[6:]

### Stat Lists

In [2]:
# Plate apperance events 
events_list = ['b1','b2','b3','hr','bb','hbp','so','fo','go','lo','po']
# Adjusted for park
events_list_adj = [f'{event}_adj' for event in events_list]
# List of statcast stats
statcast_list = ['estimated_woba_using_speedangle','to_left','to_middle','to_right','hard_hit','barrel']
# Stats for which we need to calculate rates
calc_list = ['iso','slg','obp','woba']
# Stats for which we want maximums over time
max_list = ['totalDistance','maxSpeed','maxSpin','launchSpeed']

# Stats for which we want averages over time
avg_list = events_list + statcast_list 

# All stats
stat_list = avg_list + calc_list + max_list + ['ab','pa']

### Batter
# Create position/length-specific inputs from stats
batter_avg_short = [f"{stat}_b" for stat in avg_list]
batter_avg_long =  [f"{stat}_b_long" for stat in avg_list]

batter_calc_short = [f"{stat}_b" for stat in calc_list]
batter_calc_long = [f"{stat}_b_long" for stat in calc_list]

batter_max_short = [f"{stat}_b" for stat in max_list]
batter_max_long =  [f"{stat}_b_long" for stat in max_list]

batter_stats_short = [f"{stat}_b" for stat in stat_list]
batter_stats_long  = [f"{stat}_b_long" for stat in stat_list]

# Steamer stats
batter_stats_fg =     ['b1_rate','b2_rate','b3_rate','hr_rate','bb_rate','hbp_rate','so_rate','woba','slg','obp']
# For use in imputations
batter_stats_fg_imp = batter_stats_fg + ['b_L', 'p_L']


### Pitcher
# Create position/length-specific inputs from stats
pitcher_avg_short = [f"{stat}_p" for stat in avg_list]
pitcher_avg_long =  [f"{stat}_p_long" for stat in avg_list]

pitcher_calc_short = [f"{stat}_p" for stat in calc_list]
pitcher_calc_long = [f"{stat}_p_long" for stat in calc_list]

pitcher_max_short = [f"{stat}_p" for stat in max_list]
pitcher_max_long =  [f"{stat}_p_long" for stat in max_list]

pitcher_stats_short = [f"{stat}_p" for stat in stat_list]
pitcher_stats_long  = [f"{stat}_p_long" for stat in stat_list]

# Steamer stats (for imputing)
pitcher_stats_fg =    ['H9','HR9','K9','BB9','GBrate','FBrate','LDrate','SIERA']
# Add stats for projecting pulls
pitcher_stats_fg2 = pitcher_stats_fg + ['reliability','IP_start','IP','relief_IP']
# For use in imputations
pitcher_stats_fg_imp = pitcher_stats_fg + ['b_L', 'p_L']

['b1_adj', 'b2_adj', 'b3_adj', 'hr_adj', 'bb_adj', 'hbp_adj', 'so_adj', 'fo_adj', 'go_adj', 'lo_adj', 'po_adj']


In [ ]:
# Game state
game_info_list = ['date', 'gamePk', 'atBatIndex', 'inning', 'halfInning', 'outs', 'awayscore', 'homescore']

# Player info
batter_info_list = ['batterName', 'batter', 'batSide']
pitcher_info_list = ['pitcherName', 'pitcher', 'pitchHand']
           
# PAs and ABs
pa_list = ['pa_b', 'ab_b', 'pa_p', 'ab_p', 'pa_b_long', 'ab_b_long', 'pa_p_long', 'ab_p_long']
    
# Base running
baserunning_stats = ["sba_2b", "sb_2b", "sba_3b", "sb_3b"]

### PA Model

In [ ]:
### Exclusions
# Stats that do not apply to the position or we just don't want
pa_exclude = ["maxSpeed_b", "maxSpin_b", "maxSpeed_b_long", "maxSpin_b_long", 
              "totalDistance_p", "totalDistance_p_long", "launchSpeed_p", "launchSpeed_p_long",
              "ab_b", "pa_b", "ab_b_long", "pa_b_long", 
              "ab_p", "pa_p", "ab_p_long", "pa_p_long"]

### batter_inputs
batter_inputs = batter_stats_short + batter_stats_long
batter_inputs = [item for item in batter_inputs if item not in pa_exclude]

### pitcher_inputs
pitcher_inputs = pitcher_stats_short + pitcher_stats_long
pitcher_inputs = [item for item in pitcher_inputs if item not in pa_exclude]

# Hand-specific stats (not in PA model)
batter_stats_l = [f'{stat}_l' for stat in batter_inputs]
batter_stats_r = [f'{stat}_r' for stat in batter_inputs]

pitcher_stats_l = [f'{stat}_l' for stat in pitcher_inputs]
pitcher_stats_r = [f'{stat}_r' for stat in pitcher_inputs]

# Hand dummies
hand_inputs = ['p_L', 'b_L']

# Venue ID numbers
venue_nums = ['1', '2', '3', '4', '5', '7', '10', '12', '13', '14', '15', '16', '17', '19', '22', '31', '32', 
              '680', '2392', '2394', '2395', '2535', '2536', '2602', '2680', '2681', '2701', '2735', '2756', 
              '2889', '3289', '3309', '3312', '3313', '4169', '4705', '5010', '5325', '5365', '5381', '5445']

# Venue dummies
venue_inputs = [f"venue_{num}" for num in venue_nums]

# Year dummies
year_inputs = [f"year_{year}" for year in range(2015,2025)]

# Weather inputs
weather_inputs = ['x_vect','y_vect','temperature']

# Game state inputs
game_state_inputs = ['onFirst','onSecond','onThird','top','outs_pre','score_diff']

# Imputation flag dummies
imp_inputs = ['imp_b', 'imp_p']

# Starter dummy
starter_inputs = ['starter']

# All inputs into final PA model
pa_inputs = batter_inputs + pitcher_inputs + hand_inputs + venue_inputs + year_inputs + weather_inputs + game_state_inputs + imp_inputs + starter_inputs

### Pull Inputs

In [ ]:
# List of rolling stats in pull inputs (these are derived from other status and therefore shouldn't be included in models_inputs_plus)
rolled_sums_list = [f'{stat}_sum' for stat in events_list] + ['rbi_sum', 'faced_sum']

# Pull inputs related to the game state
# OUT may be redundant, but it still might help
pull_inputs_game = ['OUT','pitcherScore','batterScore','inning_adj','outs_adj','faced_inning','br_inning','onFirst','onSecond','onThird']
pull_inputs_other = ['IP_start'] # + year_inputs

# All inputs into final pulls model (specifically, from the API/Statcast, not Steamer)
pull_inputs_pre = rolled_sums_list + pull_inputs_game + pull_inputs_other
# Elements to remove
pull_exclude = ["fo_sum", "go_sum", "lo_sum", "po_sum", "rbi_sum"] # Definitely do not want RBI sum as it's redundant with batterScore (we have no unearned runs)

# Create a new list with elements not in the elements_to_remove list
pull_inputs = [element for element in pull_inputs_pre if element not in pull_exclude]

In [ ]:
# Creating a custom MLPDropout classifier
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network._stochastic_optimizers import AdamOptimizer
from sklearn.neural_network._base import ACTIVATIONS, DERIVATIVES, LOSS_FUNCTIONS
from sklearn.utils import shuffle, gen_batches, check_random_state, _safe_indexing
from sklearn.utils.extmath import safe_sparse_dot
import warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.base import is_classifier

class MLPDropout(MLPClassifier):
    
    def __init__(
        self,
        hidden_layer_sizes=(100,),
        activation="relu",
        *,
        solver="adam",
        alpha=0.0001,
        batch_size="auto",
        learning_rate="constant",
        learning_rate_init=0.001,
        power_t=0.5,
        max_iter=200,
        shuffle=True,
        random_state=None,
        tol=1e-4,
        verbose=False,
        warm_start=False,
        momentum=0.9,
        nesterovs_momentum=True,
        early_stopping=False,
        validation_fraction=0.1,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-8,
        n_iter_no_change=10,
        max_fun=15000,
        dropout = None,
    ):
        '''
        Additional Parameters:
        ----------
        dropout : float in range (0, 1), default=None
            Dropout parameter for the model, defines the percentage of nodes
            to remove at each layer.
            
        '''
        self.dropout = dropout
        super().__init__(
            hidden_layer_sizes=hidden_layer_sizes,
            activation=activation,
            solver=solver,
            alpha=alpha,
            batch_size=batch_size,
            learning_rate=learning_rate,
            learning_rate_init=learning_rate_init,
            power_t=power_t,
            max_iter=max_iter,
            shuffle=shuffle,
            random_state=random_state,
            tol=tol,
            verbose=verbose,
            warm_start=warm_start,
            momentum=momentum,
            nesterovs_momentum=nesterovs_momentum,
            early_stopping=early_stopping,
            validation_fraction=validation_fraction,
            beta_1=beta_1,
            beta_2=beta_2,
            epsilon=epsilon,
            n_iter_no_change=n_iter_no_change,
            max_fun=max_fun,
        )
    
    def _fit_stochastic(
        self,
        X,
        y,
        activations,
        deltas,
        coef_grads,
        intercept_grads,
        layer_units,
        incremental,
    ):
        params = self.coefs_ + self.intercepts_
        if not incremental or not hasattr(self, "_optimizer"):
            if self.solver == "sgd":
                self._optimizer = SGDOptimizer(
                    params,
                    self.learning_rate_init,
                    self.learning_rate,
                    self.momentum,
                    self.nesterovs_momentum,
                    self.power_t,
                )
            elif self.solver == "adam":
                self._optimizer = AdamOptimizer(
                    params,
                    self.learning_rate_init,
                    self.beta_1,
                    self.beta_2,
                    self.epsilon,
                )

        # early_stopping in partial_fit doesn't make sense
        early_stopping = self.early_stopping and not incremental
        if early_stopping:
            # don't stratify in multilabel classification
            should_stratify = is_classifier(self) and self.n_outputs_ == 1
            stratify = y if should_stratify else None
            X, X_val, y, y_val = train_test_split(
                X,
                y,
                random_state=self._random_state,
                test_size=self.validation_fraction,
                stratify=stratify,
            )
            if is_classifier(self):
                y_val = self._label_binarizer.inverse_transform(y_val)
        else:
            X_val = None
            y_val = None

        n_samples = X.shape[0]
        sample_idx = np.arange(n_samples, dtype=int)

        if self.batch_size == "auto":
            batch_size = min(200, n_samples)
        else:
            if self.batch_size < 1 or self.batch_size > n_samples:
                warnings.warn(
                    "Got `batch_size` less than 1 or larger than "
                    "sample size. It is going to be clipped"
                )
            batch_size = np.clip(self.batch_size, 1, n_samples)

        try:
            for it in range(self.max_iter):
                if self.shuffle:
                    # Only shuffle the sample indices instead of X and y to
                    # reduce the memory footprint. These indices will be used
                    # to slice the X and y.
                    sample_idx = shuffle(sample_idx, random_state=self._random_state)

                accumulated_loss = 0.0
                for batch_slice in gen_batches(n_samples, batch_size):
                    if self.shuffle:
                        X_batch = _safe_indexing(X, sample_idx[batch_slice])
                        y_batch = y[sample_idx[batch_slice]]
                    else:
                        X_batch = X[batch_slice]
                        y_batch = y[batch_slice]
                    
                    activations[0] = X_batch
                    # (DROPOUT ADDITION) layer_units passed forward to help build dropout mask.
                    batch_loss, coef_grads, intercept_grads = self._backprop(
                        X_batch,
                        y_batch,
                        activations,
                        layer_units,
                        deltas,
                        coef_grads,
                        intercept_grads,
                    )
                    accumulated_loss += batch_loss * (
                        batch_slice.stop - batch_slice.start
                    )

                    # update weights
                    grads = coef_grads + intercept_grads
                    self._optimizer.update_params(params, grads)

                self.n_iter_ += 1
                self.loss_ = accumulated_loss / X.shape[0]

                self.t_ += n_samples
                self.loss_curve_.append(self.loss_)
                if self.verbose:
                    print("Iteration %d, loss = %.8f" % (self.n_iter_, self.loss_))

                # update no_improvement_count based on training loss or
                # validation score according to early_stopping
                self._update_no_improvement_count(early_stopping, X_val, y_val)

                # for learning rate that needs to be updated at iteration end
                self._optimizer.iteration_ends(self.t_)

                if self._no_improvement_count > self.n_iter_no_change:
                    # not better than last `n_iter_no_change` iterations by tol
                    # stop or decrease learning rate
                    if early_stopping:
                        msg = (
                            "Validation score did not improve more than "
                            "tol=%f for %d consecutive epochs."
                            % (self.tol, self.n_iter_no_change)
                        )
                    else:
                        msg = (
                            "Training loss did not improve more than tol=%f"
                            " for %d consecutive epochs."
                            % (self.tol, self.n_iter_no_change)
                        )

                    is_stopping = self._optimizer.trigger_stopping(msg, self.verbose)
                    if is_stopping:
                        break
                    else:
                        self._no_improvement_count = 0

                if incremental:
                    break

                if self.n_iter_ == self.max_iter:
                    warnings.warn(
                        "Stochastic Optimizer: Maximum iterations (%d) "
                        "reached and the optimization hasn't converged yet."
                        % self.max_iter,
                        ConvergenceWarning,
                    )
        except KeyboardInterrupt:
            warnings.warn("Training interrupted by user.")

        if early_stopping:
            # restore best weights
            self.coefs_ = self._best_coefs
            self.intercepts_ = self._best_intercepts
    
    def _backprop(self, X, y, activations, layer_units, deltas, coef_grads, intercept_grads):
        """Compute the MLP loss function and its corresponding derivatives
        with respect to each parameter: weights and bias vectors.

        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            The input data.

        y : ndarray of shape (n_samples,)
            The target values.

        activations : list, length = n_layers - 1
            The ith element of the list holds the values of the ith layer.
             
        layer_units (DROPOUT ADDITION) : list, length = n_layers
            The layer units of the neural net, this is the shape of the
            Neural Net model. This is used to build the dropout mask.

        deltas : list, length = n_layers - 1
            The ith element of the list holds the difference between the
            activations of the i + 1 layer and the backpropagated error.
            More specifically, deltas are gradients of loss with respect to z
            in each layer, where z = wx + b is the value of a particular layer
            before passing through the activation function

        coef_grads : list, length = n_layers - 1
            The ith element contains the amount of change used to update the
            coefficient parameters of the ith layer in an iteration.

        intercept_grads : list, length = n_layers - 1
            The ith element contains the amount of change used to update the
            intercept parameters of the ith layer in an iteration.

        Returns
        -------
        loss : float
        coef_grads : list, length = n_layers - 1
        intercept_grads : list, length = n_layers - 1
        """
        n_samples = X.shape[0]
        dropout_masks = None
        
        # Create the Dropout Mask (DROPOUT ADDITION)
        if self.dropout != None:
            if 0 < self.dropout < 1:
                keep_probability = 1 - self.dropout
                dropout_masks = [np.ones(layer_units[0])]
                
                # Create hidden Layer Dropout Masks
                for units in layer_units[1:-1]:
                    # Create inverted Dropout Mask, check for random_state
                    if self.random_state != None:
                        layer_mask = (self._random_state.random(units) < keep_probability).astype(int) / keep_probability
                    else:
                        layer_mask = (np.random.rand(units) < keep_probability).astype(int) / keep_probability
                    dropout_masks.append(layer_mask)
            else:
                raise ValueError('Dropout must be between zero and one. If Dropout=X then, 0 < X < 1.')
        
        # Forward propagate
        # Added dropout_makss to _forward_pass call (DROPOUT ADDITION)
        activations = self._forward_pass(activations, dropout_masks)
        
        # Get loss
        loss_func_name = self.loss
        if loss_func_name == "log_loss" and self.out_activation_ == "logistic":
            loss_func_name = "binary_log_loss"
        loss = LOSS_FUNCTIONS[loss_func_name](y, activations[-1])
        # Add L2 regularization term to loss
        values = 0
        for s in self.coefs_:
            s = s.ravel()
            values += np.dot(s, s)
        loss += (0.5 * self.alpha) * values / n_samples

        # Backward propagate
        last = self.n_layers_ - 2

        # The calculation of delta[last] here works with following
        # combinations of output activation and loss function:
        # sigmoid and binary cross entropy, softmax and categorical cross
        # entropy, and identity with squared loss
        deltas[last] = activations[-1] - y
        
        # Compute gradient for the last layer
        self._compute_loss_grad(
            last, n_samples, activations, deltas, coef_grads, intercept_grads
        )

        inplace_derivative = DERIVATIVES[self.activation]
        # Iterate over the hidden layers
        for i in range(self.n_layers_ - 2, 0, -1):
            deltas[i - 1] = safe_sparse_dot(deltas[i], self.coefs_[i].T)
            inplace_derivative(activations[i], deltas[i - 1])
            
            self._compute_loss_grad(
                i - 1, n_samples, activations, deltas, coef_grads, intercept_grads
            )
        
        # Apply Dropout Masks to the Parameter Gradients (DROPOUT ADDITION)
        if dropout_masks != None:
            for layer in range(len(coef_grads)-1):
                mask = (~(dropout_masks[layer+1] == 0)).astype(int)
                coef_grads[layer] = coef_grads[layer] * mask[None, :]
                coef_grads[layer+1] = (coef_grads[layer+1] * mask.reshape(-1, 1))
                intercept_grads[layer] = intercept_grads[layer] * mask
        
        return loss, coef_grads, intercept_grads
    
    def _forward_pass(self, activations, dropout_masks=None):
        """Perform a forward pass on the network by computing the values
        of the neurons in the hidden layers and the output layer.

        Parameters
        ----------
        activations : list, length = n_layers - 1
            The ith element of the list holds the values of the ith layer.
        dropout_mask : list, length = n_layers - 1
            The ith element of the list holds the dropout mask for the ith
            layer.
        """
        hidden_activation = ACTIVATIONS[self.activation]
        # Iterate over the hidden layers
        for i in range(self.n_layers_ - 1):
            activations[i + 1] = safe_sparse_dot(activations[i], self.coefs_[i])
            activations[i + 1] += self.intercepts_[i]
            
            # For the hidden layers
            if (i + 1) != (self.n_layers_ - 1):
                hidden_activation(activations[i + 1])
            
            # Apply Dropout Mask (DROPOUT ADDITION)
            if (i + 1) != (self.n_layers_ - 1) and dropout_masks != None:
                check1 = activations[i].copy()
                activations[i+1] = activations[i+1] * dropout_masks[i+1][None, :]

        # For the last layer
        output_activation = ACTIVATIONS[self.out_activation_]
        output_activation(activations[i + 1])
        return activations